In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1 as ff1

ff1.Cache.enable_cache('cache')

In [43]:
import warnings
warnings.filterwarnings('ignore')
# ignore info messages
import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)

In [158]:
def get_empty_dataframe():
    return pd.DataFrame(
            columns=[
                'LapStartTime',
                'LapNumber',
                'LapTime',
                'Compound',
                'TyreLife',
                'TrackStatus',
                'Stint',
                'DistanceToDriverAhead',
                'DriverAhead',
                ])

In [188]:
# Exception that is raised when there is no telemetry data for a given lap.
class NoTelemetryException(Exception):
    pass

# Find the telemetry data (Driver Ahead, Distance to Driver Ahead) for the start of a lap and merge it with the lap data.
def get_telemetry_at_start_of_lap(lap, telemetry):
    # Find the telemetry data for the start of the lap by creating a 1 second window around the lap start time.
    mask = (telemetry['Date'] > lap['LapStartDate']) & (telemetry['Date'] <= lap['LapStartDate'] + pd.Timedelta(seconds=1))
    rows = telemetry.loc[mask]
    # If there is no telemetry data for the lap, raise an exception.
    if rows.empty:
        raise NoTelemetryException("No telemetry data found for lap " + str(lap['LapNumber']) + " of " + str(lap['Driver']) + " at " + str(lap['LapStartDate']))
    row = rows.iloc[0] # There can be multiple telemetry samples in the 1 second window, so take the first one.
    telemetryInfo = row[['DriverAhead', 'DistanceToDriverAhead']] # Get the telemetry data we are interested in.
    lapInfo = lap[['LapStartTime', 'LapNumber', 'LapTime', 'Compound', 'TyreLife', 'Stint', 'TrackStatus']] # Get the lap data we are interested in.
    telemetryInfo.rename("Telemetry", inplace=True)
    lapInfo.rename("Lap", inplace=True)
    merge = pd.concat([telemetryInfo, lapInfo]) # Merge the telemetry and lap data.
    return merge


In [187]:
# Exception that is raised when there is no data for a given lap.
class NoLapException(Exception):
    pass

# Find all the laps for a given driver and merge the telemetry data for the start of each lap.
def get_laps_of_driver(driver, laps):
    driver_laps = laps.pick_driver(driver)
    if len(driver_laps['DriverNumber']) == 0:
        raise NoLapException("No laps for driver " + driver)
    driver_laps_telemetry = driver_laps.get_car_data()
    if len(driver_laps_telemetry) == 0:
        raise NoLapException("No telemetry for driver " + driver)
    try:
        driver_laps_telemetry = driver_laps_telemetry.add_driver_ahead()
        transformed_laps = []
        for index, row in driver_laps.iterrows():
            try:
                transformed_laps.append(get_telemetry_at_start_of_lap(row, driver_laps_telemetry))
            except NoTelemetryException as e:
                print(e)
    except ValueError as e:
        print("error : ", e)
        print(driver_laps_telemetry)
        return get_empty_dataframe()
    return pd.DataFrame(transformed_laps)

# Add weather data that would have been available at the start of each lap.
def add_weather_to_laps(laps, weather):
    if laps.empty:
        raise NoLapException("Laps dataframe is empty")
    # Effectuer une jointure basée sur une plage de temps
    lapsWithWeather = pd.merge_asof(laps.sort_values('LapStartTime'), 
                                    weather.sort_values('Time'), 
                                    left_on='LapStartTime', 
                                    right_on='Time', 
                                    by=None, 
                                    direction='backward')

    return lapsWithWeather

In [191]:
import os

# Get the data for all the races within a given year and save it to csv files.
def get_season_data(year, save_all_races=True):
    schedule = ff1.get_event_schedule(year, include_testing=False)
    
    path = 'data/' + str(year)
    # Create a directory for the year if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    df_season = get_empty_dataframe()
    for index, event in schedule.iterrows():
        if year == 2018 and event['RoundNumber'] < 3: # The 2 first races of 2018 do not have telemetry data
            continue
        print("Processing race round - ", event['RoundNumber'])
        race = event.get_race()
        race.load()
        df_event = get_empty_dataframe()
        for driver in race.drivers:
            print("     Processing driver - ", driver)
            try:
                df_event = pd.concat([df_event, get_laps_of_driver(driver, race.laps)], axis=0)
            except NoLapException as e:
                print(e)
    
        df_event['Track'] = event['Location']
        try:
            df_event = add_weather_to_laps(df_event, race.weather_data)
        except NoLapException as e:
            print(e)
        if save_all_races:
            # Save it to a csv file
            df_event.to_csv(path + '/' + event.EventName.replace(' ', '_').lower() + '.csv', index=False)
        df_season = pd.concat([df_season, df_event], axis=0)

    # Save the data for the whole season
    df_season.to_csv(path + '/season.csv', index=False)
    return df_season

In [ ]:
for year in range(2018, 2023):
    get_season_data(year)